# 0. Reading the Data
I will start with ride ID = 2975587283 which is the first ride of 2020 

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 100)

In [10]:
import os
os.listdir('../../data/enriched/activities/')

['.gitignore',
 '2975587283.csv',
 '2975587283.csv.csv',
 '2992935347.csv',
 '2992935347.csv.csv',
 '2994399692.csv',
 '2994399692.csv.csv',
 '3248077156.csv',
 '3248077156.csv.csv',
 '3255473930.csv',
 '3255473930.csv.csv',
 '3286667096.csv',
 '3286667096.csv.csv',
 '3290080609.csv',
 '3290080609.csv.csv',
 '3294902966.csv',
 '3294902966.csv.csv',
 '3299844988.csv',
 '3299844988.csv.csv',
 '3309212141.csv',
 '3309212141.csv.csv',
 '3311386012.csv',
 '3311386012.csv.csv',
 '3318357988.csv',
 '3318357988.csv.csv',
 '3323709534.csv',
 '3323709534.csv.csv',
 '3328174429.csv',
 '3328174429.csv.csv',
 '3338503991.csv',
 '3338503991.csv.csv',
 '3343171778.csv',
 '3343171778.csv.csv',
 '3353198447.csv',
 '3353198447.csv.csv',
 '3367098479.csv',
 '3367098479.csv.csv',
 '3367398159.csv',
 '3367398159.csv.csv',
 '3371564928.csv',
 '3371564928.csv.csv',
 '3381861723.csv',
 '3381861723.csv.csv',
 '3387969771.csv',
 '3387969771.csv.csv',
 '3392547429.csv',
 '3392547429.csv.csv',
 '3403711514.csv',


In [9]:
file_paths = {'in':'../../data/enriched/activities/2975587283.csv'
             }

df = pd.read_csv(file_paths['in'])

# guarantee the timestamps are datetime objects
df['time'] = pd.to_datetime(df['time'])

In [3]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df.info()